In [7]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()
GECKO_API = os.getenv("GECKO_API")

url = "https://api.coingecko.com/api/v3/derivatives/exchanges"

headers = {
    "accept": "application/json",
    "x-cg-api-key": GECKO_API
}

response = requests.get(url, headers=headers)
data = response.json()

# Extract just the IDs
exchange_ids = [ex["id"] for ex in data]

print("Available Exchange IDs:")
for ex_id in exchange_ids:
    print("-", ex_id)


Available Exchange IDs:
- binance_futures
- bybit
- bitget_futures
- gate_futures
- coinw_futures
- weex-futures
- hyperliquid
- okex_swap
- xt_derivatives
- bitmart_futures
- coincatch_derivatives
- huobi_dm
- mxc_futures
- toobit_derivatives
- whitebit_futures
- deepcoin_derivatives
- bingx_futures
- bitvenus
- kumex
- lbank-futures


In [8]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()
GECKO_API = os.getenv('GECKO_API')
id = 'binance_futures'

url = f"https://api.coingecko.com/api/v3/derivatives/exchanges/{id}"

headers = {"accept": "application/json" , "x-cg-api-key": GECKO_API}

response = requests.get(url, headers=headers)

print(response.text)

{"name":"Binance (Futures)","open_interest_btc":341205.1,"trade_volume_24h_btc":"891700.79","number_of_perpetual_pairs":546,"number_of_futures_pairs":31,"image":"https://coin-images.coingecko.com/markets/images/466/small/binance_futures.jpg?1706864452","year_established":2019,"country":null,"description":"","url":"https://www.binance.com/"}


In [10]:
import requests
import os
import pandas as pd
import time
from dotenv import load_dotenv

# Load API key
load_dotenv()
GECKO_API = os.getenv("GECKO_API")

BASE_URL = "https://api.coingecko.com/api/v3/derivatives/exchanges"

headers = {
    "accept": "application/json",
    "x-cg-api-key": GECKO_API
}

# --- Helper function with retry logic ---
def fetch_exchange_data(ex_id, retries=3, delay=2):
    url = f"{BASE_URL}/{ex_id}"
    for attempt in range(retries):
        try:
            r = requests.get(url, headers=headers, timeout=30)
            r.raise_for_status()
            return r.json()
        except requests.exceptions.SSLError as e:
            print(f"⚠️ SSL error on {ex_id}, retrying... ({attempt+1}/{retries})")
            time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"⚠️ Request error on {ex_id}: {e}")
            return None
    print(f"❌ Skipping {ex_id} after {retries} failed attempts.")
    return None

# 1. Get all exchange IDs
resp = requests.get(BASE_URL, headers=headers)
exchanges = resp.json()
exchange_ids = [ex["id"] for ex in exchanges]

# 2. Loop through exchanges and collect funding data
funding_data = []

for ex_id in exchange_ids:
    ex_data = fetch_exchange_data(ex_id)
    if not ex_data:
        continue  # skip failed exchange
    
    name = ex_data.get("name")
    tickers = ex_data.get("tickers", [])
    
    for t in tickers:
        if t.get("contract_type") == "perpetual":  # only perpetual contracts
            funding_data.append({
                "exchange_id": ex_id,
                "exchange_name": name,
                "symbol": t.get("symbol"),
                "base": t.get("base"),
                "target": t.get("target"),
                "funding_rate": t.get("funding_rate"),
                "open_interest_usd": t.get("open_interest_usd"),
                "h24_volume": t.get("h24_volume"),
                "trade_url": t.get("trade_url")
            })
    
    time.sleep(1)  # throttle requests to avoid SSL/rate-limit issues

# 3. Save to DataFrame
df = pd.DataFrame(funding_data)
print(df.head())

df.to_csv("funding_open_interest.csv", index=False)
print("✅ Data saved to funding_open_interest.csv")


⚠️ Request error on hyperliquid: 429 Client Error: Too Many Requests for url: https://api.coingecko.com/api/v3/derivatives/exchanges/hyperliquid
⚠️ Request error on weex-futures: 429 Client Error: Too Many Requests for url: https://api.coingecko.com/api/v3/derivatives/exchanges/weex-futures
⚠️ Request error on okex_swap: 429 Client Error: Too Many Requests for url: https://api.coingecko.com/api/v3/derivatives/exchanges/okex_swap
⚠️ Request error on xt_derivatives: 429 Client Error: Too Many Requests for url: https://api.coingecko.com/api/v3/derivatives/exchanges/xt_derivatives
⚠️ Request error on bitmart_futures: 429 Client Error: Too Many Requests for url: https://api.coingecko.com/api/v3/derivatives/exchanges/bitmart_futures
⚠️ Request error on coincatch_derivatives: 429 Client Error: Too Many Requests for url: https://api.coingecko.com/api/v3/derivatives/exchanges/coincatch_derivatives
⚠️ Request error on huobi_dm: 429 Client Error: Too Many Requests for url: https://api.coingecko.c

In [4]:
print("API Key:", GECKO_API)


API Key: CG-tEqpaCHxWu8hm5kFHc7AbC98
